# Модель "мешок слов"


Пусть есть коллекция документов $D$. Будем рассматривать модель *bag-of-words* (мешок слов), то есть каждый документ состоит из какого-то набора слов (терма) без учета их позиций внутри документа. 

Рассмотрим коллекцию документов:

In [2]:
docs = ['человек лев орел черепаха человек', 
        'лев вол орел',
        'лев черепаха лев кошка',
        'жучка кошка мышка',
        'лев орел грифон']

Посчитаем сколько каждое слово встретилось во всей коллекции и сколько каждом документе:

In [3]:
import re
import numpy as np
import pandas as pd


from collections import defaultdict, Counter 

def parse_doc(doc):
    return re.split(r'\s+', doc, re.U)

# подсчет df по коллекции
def calc_df_dict(docs):
    c = Counter() 
    for doc_id, doc in enumerate(docs):
        c.update(set(parse_doc(doc)))
    return c 
    
# подсчет tf для документа
def calc_tf_dict(doc):
    c = Counter() 
    for word in parse_doc(doc):
        c[word] += 1
    return c 
    
dfs = calc_df_dict(docs)
pd.DataFrame(data=list(dfs.items()), columns=['term', 'df'])

,term,df
0,орел,3
1,лев,4
2,человек,1
3,черепаха,2
4,вол,1
5,кошка,2
6,жучка,1
7,мышка,1
8,грифон,1


Каждому слову и каждому документу можно присвоить уникальный численный идентификатор, и построить так называемую tf-матрицу, состоящую из элементов $\{tf_{t,d}\}$ - вес слова $t$ в документе $d$ (где $t = 1 \dots n$ - индексы слов, $j = 1 \dots k$ - индексы документов). Под весом может подразумеваться число вхождений, нормализированная частота, и т.п.

### Система обозначений SMART

$\{tf_{t,d}\}$ не всегда дает адекватное представление (документы могут быть сильно не равной длины, не учитывается значимость слов). Поэтому существует более сложные модели под обобщенным названием *tf-idf* (term frequency - inverted document frequency), где используется документная частота слова. Идея в том, что для каждого слова в каждом документе считается *tf*, потом *idf* и значения перемножаются. Потом для каждого документа получается вектор, с которым можно что-то сделать (нормализовать).

| Частота термина                                                                                                  | Документная частота                                            | Нормировка                                              |
|------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------|---------------------------------------------------------|
| n $$	\text{tf}_{t,d}$$                                                                                             | n $$1$$                                                        | n $$1$$                                                 |
| l $$1 + \log{\text{tf}_{t,d}}$$                                                                                  | t $$\log{\frac{N}{\text{df}_i}}$$                              | c $$\frac{1}{\sqrt{\omega_1^2 + \ldots + \omega_m^2}}$$ |
| a $$0.5 + \frac{0.5 \text{tf}_{t,d}}{\max_t{ \text{tf}_{t,d}}}$$                                                 | p $$\text{max}(0, \log{\frac{N - \text{df}_i}{\text{df}_i}})$$ |
| b $$ 1,  \text{if }  \text{tf}_{t,d} > 0 \text{ else } 0 $$                                                              |
| L $$\frac{1 + \log{\text{tf}_{t,d}}}{1 + \log{\text{avg}_{t \in d}(\text{tf}_{t,d})}}$$                          |


### Ипользование scikit-learn

In [5]:
docs

['человек лев орел черепаха человек',
 'лев вол орел',
 'лев черепаха лев кошка',
 'жучка кошка мышка',
 'лев орел грифон']

In [11]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

vectorizer = HashingVectorizer(n_features=5)
vectorizer.fit_transform(docs).toarray()  

array([[-0.66666667,  0.66666667,  0.        , -0.33333333,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.31622777,  0.9486833 ,  0.        ,  0.        ,  0.        ],
       [ 0.57735027,  0.        ,  0.        ,  0.57735027,  0.57735027],
       [ 0.        ,  0.57735027,  0.        , -0.57735027, -0.57735027]])

In [7]:
print('\n'.join(vectorizer.get_feature_names_out()))

вол
грифон
жучка
кошка
лев
мышка
орел
человек
черепаха


## Использование в кластеризации, классификации и тематическом моделировании

Напишем код для чтения и обработки коллекции новостей

In [12]:
import gzip

from dataclasses import dataclass
from typing import Iterator

from nltk.corpus import stopwords
from yargy.tokenizer import MorphTokenizer


@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))


tokenizer = MorphTokenizer()
ru_stopwords = set(stopwords.words("russian"))


def normalize_text(text: str) -> str:
    tokens = [
        tok.normalized for tok in tokenizer(text) if tok.normalized not in ru_stopwords
    ]
    return " ".join(tokens)

In [ ]:
normalize_text("Привет, миру!")

'привет , мир !'

Прочитаем текст и преобразуем документы в вектора

In [16]:
texts = list(read_texts("data/news.txt.gz"))

In [20]:
vectorizer = TfidfVectorizer(max_df=0.2, min_df=10)
# vectorizer = HashingVectorizer()

X = vectorizer.fit_transform([normalize_text(text.text) for text in texts]).toarray()

понизим размерность c помощью `PCA`

In [24]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)
X = pca.fit_transform(X)

In [25]:
X.shape

(10000, 500)

напишем кол для преобразования новых текстов

In [26]:
def transform_text(text: str) -> np.ndarray:
    normalized_text = normalize_text(text)
    vect = vectorizer.transform([normalized_text]).toarray()
    return pca.transform(vect)

transform_text("привет миру ещё раз")

array([[-3.39188448e-02, -7.66922682e-03, -7.87604734e-03,
         1.91158993e-03,  1.09120228e-02, -1.92687864e-02,
         8.63962252e-03,  1.66701950e-02,  6.38762710e-03,
        -1.73677962e-03,  1.03452127e-02, -3.54432933e-02,
        -4.87775109e-02, -1.62290554e-02, -2.66548195e-03,
         2.27416360e-02,  4.50016370e-03, -1.31261483e-03,
        -3.46951534e-03, -1.18351487e-02,  1.56695989e-02,
        -6.16911829e-03,  2.25223226e-02, -1.49857922e-02,
        -5.50197087e-03,  3.98800374e-02,  1.25062014e-02,
        -4.69092736e-03, -5.58313037e-03,  2.57423417e-03,
         8.86044439e-03, -1.55273015e-02, -7.09154315e-04,
        -2.31184327e-03, -7.23467834e-03,  6.70779691e-04,
        -3.04841716e-02, -2.85352347e-02,  9.88397400e-06,
         1.81021102e-03,  3.76000184e-03, -1.62727651e-02,
         9.20342667e-03,  4.11565135e-02, -4.69881257e-03,
        -3.52405696e-03, -6.88625623e-03,  2.56187939e-03,
         3.33907959e-02, -2.06297923e-02,  5.13705186e-0

Кластеризация

In [27]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=10, n_init="auto")
k_means.fit(X)

,n_clusters,10
,init,'k-means++'
,n_init,'auto'
,max_iter,300
,tol,0.0001
,verbose,0
,random_state,None
,copy_x,True
,algorithm,'lloyd'


In [48]:
[(idx, texts[idx].label)  for idx, cluster in enumerate(k_means.predict(X)) if cluster == 9]

[(19, 'life'),
 (20, 'media'),
 (23, 'life'),
 (36, 'forces'),
 (39, 'forces'),
 (46, 'forces'),
 (47, 'business'),
 (62, 'sport'),
 (63, 'life'),
 (69, 'forces'),
 (75, 'business'),
 (86, 'life'),
 (93, 'forces'),
 (95, 'culture'),
 (105, 'life'),
 (112, 'life'),
 (124, 'life'),
 (129, 'life'),
 (139, 'forces'),
 (155, 'life'),
 (175, 'life'),
 (178, 'culture'),
 (180, 'media'),
 (184, 'life'),
 (186, 'forces'),
 (193, 'life'),
 (205, 'media'),
 (211, 'life'),
 (215, 'media'),
 (217, 'media'),
 (229, 'forces'),
 (231, 'business'),
 (239, 'media'),
 (247, 'forces'),
 (253, 'forces'),
 (256, 'life'),
 (258, 'media'),
 (264, 'life'),
 (274, 'life'),
 (296, 'forces'),
 (306, 'economics'),
 (317, 'life'),
 (340, 'forces'),
 (343, 'media'),
 (411, 'forces'),
 (416, 'travel'),
 (418, 'science'),
 (425, 'media'),
 (430, 'forces'),
 (435, 'media'),
 (458, 'life'),
 (464, 'life'),
 (471, 'life'),
 (483, 'media'),
 (497, 'life'),
 (499, 'forces'),
 (502, 'life'),
 (508, 'media'),
 (516, 'sport')

In [47]:
texts[62]

Text(label='sport', title='Супруга Немова рассказала о планах гимнаста разобраться со «СтопХамом»', text='Российский гимнаст, четырехкратный олимпийский чемпион Алексей Немов намерен разобраться в конфликте с активистами движения «СтопХам», с которыми у него произошла стычка. Об этом сообщает «Русская служба новостей» со ссылкой на супругу спортсмена Галину Немову.Материалы по теме13:14 16 февраля 2016«Подрался с детьми. Орал как баба»Реакция соцсетей на конфликт Алексея Немова с активистами «СтопХама»«Мы думаем, как поступить в этой ситуации, потому что такое может произойти с каждым, хочется как-то наказать этих людей. Это провокация настоящая. Они ведут себя так интеллигентно, при этом провоцируют, стучат в окно и спрашивают: "Что же вы делаете?". А что — ты даже не понимаешь в этой ситуации. Что это, Алексей тоже для себя хочет разобрать», — сказала Немова.Супруга гимнаста выразила мнение, что случившееся было провокацией. Она отметила, что ее муж просил активистов отойти от автомо

Классификация

In [49]:
from sklearn.svm import SVC

y = [text.label for text in texts]

svc = SVC()
svc.fit(X, y)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [ ]:
svc.predict(transform_text("путешествие в японию"))

array(['science'], dtype='<U9')

Тематическое моделирование

In [68]:
import re

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel 

normalized_tokens = [
    re.findall(r"\b\w+\b", normalize_text(text.text))
    for text in texts
]
dictionary = Dictionary(normalized_tokens)

corpus = [dictionary.doc2bow(text) for text in normalized_tokens]

lda = LdaModel(corpus, num_topics=10)

In [83]:
id2token = {v : k for (k, v) in dictionary.token2id.items()}
[(id2token[token_id], p) for (token_id, p) in lda.get_topic_terms(9, 20)]

[('который', 0.00830321),
 ('год', 0.0054368204),
 ('это', 0.004838807),
 ('компания', 0.0043977294),
 ('сообщать', 0.0036527005),
 ('также', 0.0031273172),
 ('полиция', 0.0029178592),
 ('доллар', 0.002819416),
 ('время', 0.00247647),
 ('мочь', 0.0024313491),
 ('миллион', 0.0022946375),
 ('свой', 0.0022523808),
 ('реклама', 0.0022084913),
 ('самолёт', 0.0022002487),
 ('мечело', 0.002067581),
 ('сотрудник', 0.0020038143),
 ('однако', 0.0019964536),
 ('стать', 0.001971077),
 ('около', 0.001962201),
 ('кипр', 0.0018835498)]